In [1]:
import sys
print(sys.executable)
print(sys.prefix)

G:\GeekDS\venv\python.exe
G:\GeekDS\venv


In [2]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	./

nothing added to commit but untracked files present (use "git add" to track)


In [6]:
!git remote -v

origin	https://github.com/Lavishgangwani/DataScienceGeek.git (fetch)
origin	https://github.com/Lavishgangwani/DataScienceGeek.git (push)


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split , GridSearchCV , RandomizedSearchCV
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder

## Used Car Price Prediction
1) Problem statement.
    - This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
    - If user can predict the price of the car based on input features.
    - Prediction results can be used to give new seller the price suggestion based on market condition.
2) Data Collection.
    - The Dataset is collected from scrapping from cardheko webiste
    - The data consists of 13 column and 15411 rows.

In [12]:
#loading the data
df = pd.read_csv('cardekho_imputated.csv' , index_col=[0])
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [13]:
df.shape

(15411, 13)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15411 entries, 0 to 19543
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_name           15411 non-null  object 
 1   brand              15411 non-null  object 
 2   model              15411 non-null  object 
 3   vehicle_age        15411 non-null  int64  
 4   km_driven          15411 non-null  int64  
 5   seller_type        15411 non-null  object 
 6   fuel_type          15411 non-null  object 
 7   transmission_type  15411 non-null  object 
 8   mileage            15411 non-null  float64
 9   engine             15411 non-null  int64  
 10  max_power          15411 non-null  float64
 11  seats              15411 non-null  int64  
 12  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1.6+ MB


In [15]:
#checking null values
df.isnull().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [38]:
#drop unnecessary cols
df.drop(columns=['car_name' , 'brand'] , axis=1 , inplace = True)
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [41]:
cat_cols = df.select_dtypes(include='O')
for i in cat_cols.columns:
    print(f'Value Counts of {i}')
    print(cat_cols[i].value_counts())
    print('-----'*10)

Value Counts of model
model
i20            906
Swift Dzire    890
Swift          781
Alto           778
City           757
              ... 
Ghibli           1
Altroz           1
GTC4Lusso        1
Aura             1
Gurkha           1
Name: count, Length: 120, dtype: int64
--------------------------------------------------
Value Counts of seller_type
seller_type
Dealer              9539
Individual          5699
Trustmark Dealer     173
Name: count, dtype: int64
--------------------------------------------------
Value Counts of fuel_type
fuel_type
Petrol      7643
Diesel      7419
CNG          301
LPG           44
Electric       4
Name: count, dtype: int64
--------------------------------------------------
Value Counts of transmission_type
transmission_type
Manual       12225
Automatic     3186
Name: count, dtype: int64
--------------------------------------------------


In [54]:
#now extraction columns types
cat_features = [i for i in df.columns if df[i].dtypes == 'O']
print("Categorical Features : ",len(cat_features))

num_features = [i for i in df.columns if df[i].dtypes != 'O']
print("Numerical Features : ",len(num_features))

descrete_num_features = [i for i in num_features if df[i].nunique() <= 25] 
print("Descrete Numerical Features : ",len(descrete_num_features))

continous_num_features = [i for i in num_features if df[i].nunique() >= 25]
print("Continuous Numerical Features : ",len(continous_num_features))

Categorical Features :  4
Numerical Features :  7
Descrete Numerical Features :  2
Continuous Numerical Features :  5


In [61]:
#splitt the data
X = df.drop(columns=['selling_price'] , axis=1)
y = df['selling_price']

### Feature Encoding and Scaling
- __One Hot Encoding for Columns which had lesser unique values and not ordinal__

- __One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.__

In [62]:
le = LabelEncoder()
X['model']= le.fit_transform(X['model'])

In [63]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [69]:
# Create Column Transformer with 3 types of transformers
cat_feat = X.select_dtypes(include='object').columns
num_feat = X.select_dtypes(exclude='object').columns

st = StandardScaler()
ohe_ = OneHotEncoder()

from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('OHE',ohe_,cat_feat),
    ('StScaler',st,num_feat)
],remainder='passthrough')

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('OHE', OneHotEncoder(),
                                 Index(['seller_type', 'fuel_type', 'transmission_type'], dtype='object')),
                                ('StScaler', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])